In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-10-28"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
9493,2024-10-28,Brasil Nbb,19:30,Paulistano,Bauru,1.41,2.62,148.5,1.80,1.86,-6.5,1.94,1.74,EyErNBHl,0.709220,0.381679,0.555556,0.537634,0.090899,0.6,0.4,NaN,NaN,133.574,38.667884,0.289487,158.750,33.126424,0.208670,136.418,165.866,173.70,184.96,0.0,0.0,0.0,0.0,0.424615,0.023184,0.076859,-2.52,-0.504,-0.813492,0.650272,0.4,-0.250272,-0.17,-0.034,-47.647059,0.503248,0.5,-0.003248
9494,2024-10-28,Brasil Nbb,20:00,Franca,Pato,1.10,5.39,156.5,1.83,1.83,-12.5,1.92,1.75,M1HjLkn1,0.909091,0.185529,0.546448,0.546448,0.094620,0.0,0.2,NaN,NaN,125.782,38.908994,0.309337,163.246,36.198270,0.221741,116.412,234.500,85.32,225.17,0.0,0.0,0.0,0.0,0.934819,0.000000,0.065509,0.51,0.102,0.980392,0.825726,0.9,0.074274,0.87,0.174,25.229885,0.412325,0.5,0.087675
9495,2024-10-28,Eua Nba,20:00,Orlando Magic,Indiana Pacers,1.45,2.90,222.5,1.80,2.00,-6.5,2.02,1.83,f5M2bSsT,0.689655,0.344828,0.555556,0.500000,0.034483,0.6,0.6,NaN,NaN,180.200,41.952232,0.232809,191.368,43.071595,0.225072,168.386,302.082,135.72,265.58,0.0,0.0,0.0,0.0,0.471405,0.074432,0.069792,0.78,0.156,2.884615,0.623141,0.6,-0.023141,-1.37,-0.274,-6.934307,0.493238,0.3,-0.193238
9496,2024-10-28,Eua Nba,20:30,Boston Celtics,Milwaukee Bucks,1.22,4.55,232.5,1.83,2.01,-9.5,1.93,1.93,6qjLG9cc,0.819672,0.219780,0.546448,0.497512,0.039452,1.0,0.6,NaN,NaN,157.898,24.113250,0.152714,159.592,29.406532,0.184261,202.066,173.690,192.72,121.38,0.0,0.0,0.0,0.0,0.816175,0.066291,0.000000,-0.81,-0.162,-1.358025,0.733438,0.8,0.066562,-3.37,-0.674,-5.267062,0.483700,0.1,-0.383700
9497,2024-10-28,Eua Nba,20:30,Miami Heat,Detroit Pistons,1.27,4.05,219.5,1.83,2.02,-8.5,1.96,1.91,Qo6MEmS9,0.787402,0.246914,0.546448,0.495050,0.034315,0.6,0.2,NaN,NaN,210.770,65.286405,0.309752,402.302,224.735147,0.558623,264.818,266.836,175.57,511.06,0.0,1.0,0.0,0.0,0.739006,0.069792,0.018271,1.16,0.232,1.163793,0.474309,0.5,0.025691,-0.64,-0.128,-23.828125,0.307433,0.4,0.092567
9498,2024-10-28,Eua Nba,20:30,New York Knicks,Cleveland Cavaliers,1.72,2.20,221.5,1.85,1.99,-3.5,2.02,1.83,WpYgduCr,0.581395,0.454545,0.540541,0.502513,0.035941,0.4,0.8,NaN,NaN,193.630,70.332859,0.363233,181.428,34.162921,0.188300,184.144,195.062,184.50,159.30,0.0,0.0,0.0,0.0,0.173169,0.051560,0.069792,3.11,0.622,1.157556,0.621331,0.8,0.178669,-2.41,-0.482,-2.489627,0.537035,0.2,-0.337035
9499,2024-10-28,Eua Nba,20:30,Toronto Raptors,Denver Nuggets,3.60,1.32,219.5,1.93,1.93,6.5,1.96,1.75,hxzXnxYF,0.277778,0.757576,0.518135,0.518135,0.035354,0.6,0.4,NaN,NaN,366.166,176.539327,0.482129,173.768,36.918452,0.212458,251.616,190.314,290.95,212.52,0.0,0.0,0.0,0.0,0.655367,0.000000,0.080050,3.77,0.754,3.448276,0.493721,0.5,0.006279,-1.12,-0.224,-1.428571,0.529567,0.5,-0.029567
9500,2024-10-28,Eua Nba,21:00,Memphis Grizzlies,Chicago Bulls,1.35,3.40,231.5,1.80,1.99,-8.5,2.04,1.81,h0M2fJte,0.740741,0.294118,0.555556,0.502513,0.034858,0.6,0.6,NaN,NaN,248.140,78.009615,0.314377,388.740,122.486544,0.315086,205.330,328.564,153.22,605.15,0.0,1.0,0.0,0.0,0.610345,0.070897,0.084485,-3.32,-0.664,-0.527108,0.531100,0.2,-0.331100,1.91,0.382,6.282723,0.425452,0.5,0.074548
9501,2024-10-28,Eua Nba,21:00,San Antonio Spurs,Houston Rockets,2.27,1.68,220.5,1.87,1.97,2.5,1.95,1.91,0QPAhcB7,0.440529,0.595238,0.534759,0.507614,0.035767,0.4,0.6,NaN,NaN,269.894,74.389023,0.275623,187.846,27.957240,0.148831,208.718,192.126,204.92,188.68,0.0,0.0,0.0,0.0,0.211237,0.036828,0.014655,2.31,0.462,2.748918,0.491264,0.6,0.1087

## Apostas do dia

### Over v0 (LDA)

In [9]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
0,20:30,Eua Nba,Boston Celtics,Milwaukee Bucks,232.5,1.83,0.9811,Over
1,20:30,Eua Nba,Miami Heat,Detroit Pistons,219.5,1.83,0.9928,Over
2,20:30,Eua Nba,New York Knicks,Cleveland Cavaliers,221.5,1.85,0.9018,Over
3,21:00,Eua Nba,San Antonio Spurs,Houston Rockets,220.5,1.87,0.9837,Over
4,21:30,Eua Nba,Dallas Mavericks,Utah Jazz,233.5,1.87,0.9882,Over
5,20:30,Eua Nba,Atlanta Hawks,Washington Wizards,232.5,1.88,0.9731,Over
